## **Setting up Development Environment**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [4]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 3.4 MB 12.6 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 3.3 MB 43.4 MB/s 
     |████████████████████████████████| 61 kB 541 kB/s 
     |████████████████████████████████| 895 kB 51.9 MB/s 
     |████████████████████████████████| 1.2 MB 9.1 MB/s 


## **Importing Libraries**

In [5]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import RobertaTokenizer, RobertaModel

import pickle

## **Loading Data**

In [6]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [7]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [8]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam2541,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [9]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [10]:
dataframe.shape

(5880, 3)

In [11]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [12]:
dataframe.shape

(5877, 3)

In [13]:
text = dataframe['text']

In [14]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [15]:
label = dataframe['category']

In [16]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [17]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [18]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

## **Feature Extraction**

In [19]:
# Load Transformer Model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [21]:
print(tokenized_input)

{'input_ids': tensor([[    0, 12798,  2840, 10470, 12798,  2840,  9470, 12798,  2840,  7487,
         12798, 10965, 10172, 12798,  2840,  2840, 12798,  2840, 14292, 12798,
         10965, 23133,  1437, 12798,  2840, 11423, 12798,  2840,  4726, 12798,
          2840, 10965, 12798,  2840, 15722, 12798, 10965,  8384, 12798,  2840,
         15722, 12798,  2840,  7487, 12798, 10965,  8384,  1437, 12798,  2840,
         10278, 12798, 10965,  6382, 12798,  2840,  7487, 12798,  2840,  8906,
         12798, 10965, 23133,  1437, 12798,  2840, 15113, 12798,  2840,  4726,
         12798,  2840,  7487, 12798, 10965,  8384, 12798,  2840, 10278, 12798,
          2840,  4726, 12798,  2840, 15722,  1437, 12798,  2840, 10278, 12798,
          2840,  4333, 12798,  2840,  2840, 12798, 10965,  8384,  1437, 12798,
          2840,  8906, 12798, 10965, 27819, 12798,  2840, 15389, 12798, 10965,
          8384, 12798,  2840, 15389, 12798,  2840,  9470,  1437, 12798,  2840,
         10278, 12798, 10965, 27819, 1

In [22]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0063,  0.0794, -0.0413,  ..., -0.0326, -0.0405, -0.0223],
         [-0.1587, -0.4954, -0.1216,  ..., -0.0949,  0.0450,  0.1873],
         [-0.0410,  0.1313, -0.1700,  ..., -0.0965,  0.1742,  0.2042],
         ...,
         [ 0.2087, -0.2437,  0.0554,  ..., -0.0412, -0.2684,  0.4072],
         [ 0.1189, -0.1301,  0.1034,  ...,  0.3891,  0.2017,  0.2436],
         [ 0.0078,  0.0743, -0.0805,  ..., -0.0624, -0.0360, -0.0457]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 2.1562e-02, -1.9498e-01, -2.0079e-01, -6.8163e-02,  1.2238e-01,
          2.0125e-01,  2.6808e-01, -1.1759e-01, -1.3763e-01, -1.9818e-01,
          2.7310e-01, -4.7037e-03, -1.4203e-01,  1.2373e-01, -1.4511e-01,
          4.8426e-01,  1.9507e-01, -4.9102e-01, -3.5521e-03, -3.7451e-02,
         -2.5936e-01,  5.8937e-02,  4.6942e-01,  3.6696e-01,  1.3080e-01,
          3.6175e-02, -1.8173e-01, -1.3128e-02,  1.9493e-01,  2.673

In [23]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [24]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [25]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./XLM-Roberta-No-Stem-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./XLM-Roberta-No-Stem-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./XLM-Roberta-No-Stem-Embedded-TrainData/output1.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output2.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output3.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output4.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output5.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output6.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output7.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output8.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output9.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output10.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output11.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output12.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output13.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output14.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output15.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output16.pickle done
./XLM-Roberta-No-

In [26]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./XLM-Roberta-No-Stem-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./XLM-Roberta-No-Stem-Embedded-TrainData/output1.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output2.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output3.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output4.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output5.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output6.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output7.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output8.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output9.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output10.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output11.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output12.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output13.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output14.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output15.pickle done
./XLM-Roberta-No-Stem-Embedded-TrainData/output16.pickle done
./XLM-Roberta-No-

In [27]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [28]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [29]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
X_train.shape

(4701, 768)

In [32]:
X_val.shape

(1176, 768)

## **Logistic Regression**

In [33]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0

## **Support Vector Machine**

In [34]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [35]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.84      0.67      0.75       950
           1       0.25      0.46      0.33       226

    accuracy                           0.63      1176
   macro avg       0.55      0.57      0.54      1176
weighted avg       0.73      0.63      0.67      1176



## **Stochastic Gradient Descent**

In [36]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.40      0.01      0.02       226

    accuracy                           0.81      1176
   macro avg       0.60      0.50      0.46      1176
weighted avg       0.73      0.81      0.72      1176



## **K Nearest Neighbours**

In [37]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.82      0.93      0.87       950
           1       0.32      0.14      0.19       226

    accuracy                           0.78      1176
   macro avg       0.57      0.53      0.53      1176
weighted avg       0.72      0.78      0.74      1176



## **Decision tree**

In [38]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83       950
           1       0.30      0.32      0.31       226

    accuracy                           0.73      1176
   macro avg       0.57      0.57      0.57      1176
weighted avg       0.73      0.73      0.73      1176



## **Random Forest**

In [39]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89       950
           1       0.55      0.05      0.10       226

    accuracy                           0.81      1176
   macro avg       0.68      0.52      0.50      1176
weighted avg       0.76      0.81      0.74      1176



## **Majority Voting**

In [40]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Using only KNN, DT & RF in predictions due to poor performance of other models
    predictions = [knn_pred[i], dt_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))    

              precision    recall  f1-score   support

           0       0.82      0.97      0.89       950
           1       0.50      0.11      0.18       226

    accuracy                           0.81      1176
   macro avg       0.66      0.54      0.53      1176
weighted avg       0.76      0.81      0.75      1176



## **Loading Testing Data**

In [41]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [42]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_245,கலகலப்பு அப்புறம் . நான் ரசிச்சு சிரிச்சு ஆழுத...
freq,1,1


In [43]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [44]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

## **Test Feature Extraction**

In [45]:
# Load Transformer Model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [47]:
print(tokenized_input)

{'input_ids': tensor([[    0, 12798,  2840, 10470, 12798,  2840, 14292, 12798, 10965, 23133,
         12798,  2840,  8906, 12798,  2840,  4726, 20857,  1437, 12798,  2840,
          6382, 12798,  2840, 10470, 12798, 10965, 10172, 12798,  2840, 15722,
         12798, 10965,  8384, 12798,  2840, 15722, 12798, 10965, 10172, 12798,
          2840, 10470, 12798,  2840,  4726, 12798,  2840, 15375, 12798, 10965,
          8384,  1437, 12798,  2840, 15722, 12798,  2840, 10470, 12798, 10965,
         10172, 12798,  2840,  7487, 12798, 10965, 10172, 12798,  2840, 11423,
         12798, 10965,  8384, 12798,  2840, 10470, 12798, 10965, 13859, 12798,
          2840,  2840, 12798, 10965,  8384,  1437, 12798,  2840,  2840, 12798,
          2840,  4726, 12798,  2840, 18537, 12798, 10965,  8384,  1437, 12798,
          2840, 10470, 12798,  2840, 14292, 12798, 10965, 23133, 12798,  2840,
          8906, 12798,  2840,  4726,  3392,  1343,   740, 38635,  1437, 12798,
          2840,  4333, 12798, 10965,  

In [48]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0090,  0.0436, -0.0383,  ..., -0.0351, -0.0521, -0.0062],
         [-0.1702, -0.5530, -0.1372,  ..., -0.1650,  0.0284,  0.1762],
         [-0.0318,  0.1725, -0.2036,  ..., -0.1959,  0.1403,  0.2386],
         ...,
         [ 0.2144, -0.2416, -0.0509,  ..., -0.0717, -0.3014,  0.3329],
         [ 0.1936, -0.1306,  0.1359,  ...,  0.4561,  0.1976,  0.2199],
         [ 0.0075,  0.0362, -0.0778,  ..., -0.0733, -0.0464, -0.0274]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 1.9013e-02, -1.9538e-01, -1.9757e-01, -7.6823e-02,  1.1885e-01,
          1.9734e-01,  2.6451e-01, -1.3043e-01, -1.3822e-01, -1.9336e-01,
          2.5696e-01, -5.6764e-03, -1.3216e-01,  1.0346e-01, -1.2139e-01,
          4.8883e-01,  1.9706e-01, -4.8506e-01, -5.7173e-03, -3.1739e-02,
         -2.5866e-01,  6.9922e-02,  4.6872e-01,  3.6401e-01,  1.3497e-01,
          4.7366e-02, -1.7746e-01, -1.2801e-02,  1.7901e-01,  2.500

In [49]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [50]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [51]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./XLM-Roberta-No-Stem-Embedded-TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./XLM-Roberta-No-Stem-Embedded-TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./XLM-Roberta-No-Stem-Embedded-TestData/output1.pickle done
./XLM-Roberta-No-Stem-Embedded-TestData/output2.pickle done
./XLM-Roberta-No-Stem-Embedded-TestData/output3.pickle done
./XLM-Roberta-No-Stem-Embedded-TestData/output4.pickle done


In [52]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./XLM-Roberta-No-Stem-Embedded-TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./XLM-Roberta-No-Stem-Embedded-TestData/output1.pickle done
./XLM-Roberta-No-Stem-Embedded-TestData/output2.pickle done
./XLM-Roberta-No-Stem-Embedded-TestData/output3.pickle done
./XLM-Roberta-No-Stem-Embedded-TestData/output4.pickle done


In [53]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [54]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [55]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [56]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [knn_test_pred[i], dt_test_pred[i], rf_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [57]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [58]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [59]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.88      0.68      0.77       536
           1       0.29      0.58      0.39       118

    accuracy                           0.66       654
   macro avg       0.58      0.63      0.58       654
weighted avg       0.77      0.66      0.70       654



In [62]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.50      0.01      0.02       118

    accuracy                           0.82       654
   macro avg       0.66      0.50      0.46       654
weighted avg       0.76      0.82      0.74       654



In [63]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.83      0.95      0.89       536
           1       0.33      0.11      0.16       118

    accuracy                           0.80       654
   macro avg       0.58      0.53      0.52       654
weighted avg       0.74      0.80      0.76       654



In [64]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.85      0.81      0.83       536
           1       0.29      0.34      0.31       118

    accuracy                           0.73       654
   macro avg       0.57      0.58      0.57       654
weighted avg       0.75      0.73      0.74       654



In [65]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.82      0.99      0.90       536
           1       0.45      0.04      0.08       118

    accuracy                           0.82       654
   macro avg       0.64      0.52      0.49       654
weighted avg       0.76      0.82      0.75       654



In [66]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.82      0.98      0.89       536
           1       0.32      0.05      0.09       118

    accuracy                           0.81       654
   macro avg       0.57      0.51      0.49       654
weighted avg       0.73      0.81      0.75       654



## **Saving the Predictions**

In [67]:
# Logistic Regression

predictions = le.inverse_transform(lr_test_pred)

dataframe_test_lr = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_lr = pd.DataFrame(dataframe_test_lr)
dataframe_test_lr.to_csv("Output/XLM-Roberta-No-Stem-LogisticRegression.csv")
dataframe_test_lr

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [68]:
# Support Vector Machine

predictions = le.inverse_transform(svm_test_pred)

dataframe_test_svm = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_svm = pd.DataFrame(dataframe_test_svm)
dataframe_test_svm.to_csv("Output/XLM-Roberta-No-Stem-SupportVectorMachine.csv")
dataframe_test_svm

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [69]:
# Naive Bayes

predictions = le.inverse_transform(nb_test_pred)

dataframe_test_nb = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_nb = pd.DataFrame(dataframe_test_nb)
dataframe_test_nb.to_csv("Output/XLM-Roberta-No-Stem-NaiveBayes.csv")
dataframe_test_nb

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,OFF
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",OFF
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [70]:
# Stochastic Gradient Descent

predictions = le.inverse_transform(sgd_test_pred)

dataframe_test_sgd = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_sgd = pd.DataFrame(dataframe_test_sgd)
dataframe_test_sgd.to_csv("Output/XLM-Roberta-No-Stem-StochasticGradientDescent.csv")
dataframe_test_sgd

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [71]:
# K Nearest Neighbours

predictions = le.inverse_transform(knn_test_pred)

dataframe_test_knn = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_knn = pd.DataFrame(dataframe_test_knn)
dataframe_test_knn.to_csv("Output/XLM-Roberta-No-Stem-KNearestNeighbours.csv")
dataframe_test_knn

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,OFF
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [72]:
# Decision Tree

predictions = le.inverse_transform(dt_test_pred)

dataframe_test_dt = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_dt = pd.DataFrame(dataframe_test_dt)
dataframe_test_dt.to_csv("Output/XLM-Roberta-No-Stem-DecisionTree.csv")
dataframe_test_dt

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,OFF
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [73]:
# Random Forest

predictions = le.inverse_transform(rf_test_pred)

dataframe_test_rf = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_rf = pd.DataFrame(dataframe_test_rf)
dataframe_test_rf.to_csv("Output/XLM-Roberta-No-Stem-RandomForest.csv")
dataframe_test_rf

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [74]:
# Majority Voting

predictions = le.inverse_transform(mv_test_pred)

dataframe_test_mv = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_mv = pd.DataFrame(dataframe_test_mv)
dataframe_test_mv.to_csv("Output/XLM-Roberta-No-Stem-MajorityVoting.csv")
dataframe_test_mv

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT
